# 2-C Identify Idioms
- author: Eu-Bin KIM @likelion
- 14th of August 2021
- tlrndk123@gmail.com


## To-do's
- `build_patterns`
- `build_patterns_list_comp`


In [ ]:
# spacy 라이브러리 설치
# https://spacy.io/
!pip3 install spacy
# 사전훈련된 nlp 모델 다운로드 
# https://spacy.io/models/en#en_core_web_sm
!python3 -m download en_core_web_sm

In [2]:
# --- the libraries needed --- #
import spacy
from spacy.matcher import Matcher
from typing import List, Dict, Tuple

In [3]:
# --- global constants & variables --- #
# 이번숙제의 목표는, 여려 용례로부터 다음 두 관용구를 검출해내는 것입니다.
IDIOMS = [
          # 경제적으로 독립하다, 라느 뜻의 관용구.
          # https://idioms.thefreedictionary.com/stand+on+one%27s+own+feet
          "stand on one's own feet",
          # "눈을 뜨다"; 시야를 넓히다. 
          # https://www.merriam-webster.com/dictionary/open%20one%27s%20eyes
          "open one's eyes"
]

# 다음 용례로부터 해당 관용구를 검출해내면 됩니다:
# (입력, 정답) = (용례, 관용구)로 설정하도록 하겠습니다. 그래서 list of tuples 입니다.                            
BATCH: List[Tuple[str , str]] = [
    # --- stand on one's own feet의 용례 --- #
    ("if you don't want to do the chores, move out and stand on your own feet!", IDIOMS[0]), # one's -> your
    ("It's difficult for students to stand on their own feet without help from their parents", IDIOMS[0]), # one's -> their
    ("I've been standing on my own feet since I was sixteen years old",  IDIOMS[0]), # stand -> standing, one's -> my
    # --- open one's eyes의 용례 --- #
    ("the letter finally opened my eyes to the truth", IDIOMS[1]),  # open -> opended, one's -> my
    ("The documentary really opened her eyes to the conditions in that country", IDIOMS[1]), # open -> opended, one's -> her
]
# natural language pipeline (nlp) 로드하기
nlp = spacy.load("en_core_web_sm")
# stand on one's own feet으로부터, 소유격이 들어가야 하는 부분은 one's 로 표기될 것임을 알 수 있습니다.
# 나중에 키워드로 사용하기 위해 여기에 미리 상수로 정의해둘게요.
PRP_PLACEHOLDER = "one's"

**Question: `spacy` 의 `nlp`는 뭐죠? 이걸로 무엇을 할 수 있나요?**

> Answer: 토큰화, 표제어 추출, 품사 추출 등의 작업을 매우 빠르게, 한번에 처리할 수 있는 SpaCy를 대표하는 파이프라인 입니다.
- `token.text`: 토큰화된 결과 확인
- `token.lemma_`: 표제어 추출 결과 확인
- `token.pos_`: coarse 품사 추출 결과 확인
- `token.tag_`: fine-grained 품사 추출 결과 확인
- `token.is_stop` : 불용어인지 아닌지?
- `token.is_punct` : punctuation인지 아닌지?
- 이외에도 더 많은 attributes를 접근할 수 있으며, 접근 가능한
 모든 attributes의 리스트는 [이 문서](https://spacy.io/api/token#attributes)에서 확인가능합니다. 
- natural language pipeline에 대한 더 자세한 설명은 [이 문서](https://spacy.io/api#architecture-pipeline)를 참조하세요. 

In [ ]:
# orth_ 불렀던 것 = 토큰화된 토큰. 
# text. = 그렇게 어려운 말 쓰지말고, 그냥 text라고 부르자.

In [6]:
# 한번 배치 속 첫번째 문서를 nlp로 처리해볼까요?
sent = BATCH[0][0]  
attrs = [
           # (토큰, 추출된 표제어, 추출된 품사(coarse), 추출된 더 자세한 품사(fine-grained))
           (token.text, token.lemma_, token.pos_, token.tag_)
           for token in nlp(sent)
            # nlp(sent) -> Doc -> 루프 -> Token.
            # list comprehension 과 궁합이 잘맞는 nlp (필터를 걸기 쉽다)
            # if not token.is_stop
            # if not token.is_punct
]
for info in attrs:
  print(info)

('if', 'if', 'SCONJ', 'IN')
('you', '-PRON-', 'PRON', 'PRP')
('do', 'do', 'AUX', 'VBP')
("n't", 'not', 'PART', 'RB')
('want', 'want', 'VERB', 'VB')
('to', 'to', 'PART', 'TO')
('do', 'do', 'AUX', 'VB')
('the', 'the', 'DET', 'DT')
('chores', 'chore', 'NOUN', 'NNS')
(',', ',', 'PUNCT', ',')
('move', 'move', 'VERB', 'VB')
('out', 'out', 'ADV', 'RB')
('and', 'and', 'CCONJ', 'CC')
('stand', 'stand', 'VERB', 'VB')
('on', 'on', 'ADP', 'IN')
('your', '-PRON-', 'DET', 'PRP$')
('own', 'own', 'ADJ', 'JJ')
('feet', 'foot', 'NOUN', 'NNS')
('!', '!', 'PUNCT', '.')


**Question:  `token.pos_`, `token.tag_`에 담겨 있는 기호의 의미가 무엇인가요?**

> Answer: `spacy.explain()` 함수를 사용하면, 각 태그가 무엇을 뜻하는지 확인해볼수 있습니다.
- `token.pos_` = coarse(일반적인) 품사: [Universal Part of Speech](https://universaldependencies.org/u/pos/) 기반
- `token.tag_` = fine-grained(구체적인) 품사: [Penn Tree bank](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) 기반
- [spacy 공식 문서](https://spacy.io/usage/linguistic-features#pos-tagging)에서 몇몇 `token.pos_`의 예시를 소개해줍니다.
- 지원하는 모든 품사 태그는 [공식 리포의 이 부분](https://github.com/explosion/spaCy/blob/cc5aeaed29c067f60d11e07496704406a1577a35/spacy/glossary.py#L17-L97)에서 확인 가능합니다.


In [5]:
print(spacy.explain("PRP"))  #  e.g. you, them, I
print(spacy.explain("PRP$"))  # e.g. your, their, my
print(spacy.explain("AUX")) # e.g. auxilary = 보"조"하는 이라는 뜻의 형용사입니다. 즉 AUX는 "조"동사를 의미합니다: do, can, will 
print(spacy.explain("SCONJ")) # e.g. if, because, when 

pronoun, personal
pronoun, possessive
auxiliary
subordinating conjunction


그런 품사 태그를 활용하면 관용구를 검출할 수 있는 패턴을 정의할 수 있습니다. 어떻게 할 수 있을까요? *stand on **one's** own feet*, *open **one's** eyes* 모두 ***one's*** 라는, 인칭 대명사의 소유격 형태를 요구합니다 (e.g. my, her, him, their). 그렇다면, 각 관용구의 검출 패턴을 다음과 같이 품사(POS)로 정의하면, `one's`의 여러 변화형에 대응할 수 있을 것입니다:
 - `stand on [POS=인칭소유격] own feet`
 - `open [POS=인칭소유격] eyes`
 
이를 위해선 `one's`라는 토큰이 위치하는 자리에 해당하는 패턴을 만들어줘야 합니다.

우선, `nlp`로 각 관용구를 토크나이즈를 했을 때, 바라는대로 `one's`라는 토큰이 출력되는지 보겠습니다:

In [7]:
for idiom in IDIOMS: 
  print(list(nlp(idiom)))

[stand, on, one, 's, own, feet]
[open, one, 's, eyes]


문제가 보입니다. `one's`가 하나의 토큰으로 존재하지 않고, `one` , `'s`로 나뉘어집니다. 

두 토큰으로 나누지 않고 `one's`를 하나의 토큰으로 취급하기 위해,  [`tokenizer.add_special_case`](https://spacy.io/api/tokenizer#add_special_case) 함수를 사용하여 다음과 같이 
새로운 토큰화 규칙을 추가해줍니다.


In [8]:
SPECIAL_CASE = [{"ORTH": "one's"}]
nlp.tokenizer.add_special_case(PRP_PLACEHOLDER, SPECIAL_CASE)
for idiom in IDIOMS: 
  # 이제 one's를 하나의 토큰으로 취급할 것 입니다.
  print(list(nlp(idiom)))

[stand, on, one's, own, feet]
[open, one's, eyes]


**Question: spacy의 `Matcher`로는 무엇을 할 수 있나요?**
> Answer: 정규표현식과 비슷한 기능을 합니다. 다만 Matcher로는 표제어(LEMMA), 품사(POS)로 패턴을 정의하여 보다 더 정교한 패턴을 편리하게 정의할 수 있습니다. 
- spacy의 `Matcher`에 대한 더 자세한 내용은 [이 문서](https://spacy.io/api/matcher)에서 확인할 수 있습니다.

In [9]:
# NORM이란 정규화 된 폼
# 표제어와 거의 비슷한 용도로 사용되지만 표제어 != 정규화 된 폼인 경우가 있다
# ex) gonna는 gon, na로 쪼개지는데
# go의 표제어는 go이지만, 정규화 된 폼은 going이다

case = [{"ORTH":"gon", "LEMMA":"go", "NORM":"going"}, {"ORTH": "na", "LEMMA": "to"}]
nlp.tokenizer.add_special_case("gonna", case)
tokens = [
          (token.text, token.lemma_, token.norm_)
          for token in nlp("I'm gonna to it")
]
print(tokens)

[('I', '-PRON-', 'i'), ("'m", 'be', 'am'), ('gon', 'go', 'going'), ('na', 'to', 'na'), ('to', 'to', 'to'), ('it', '-PRON-', 'it')]


In [18]:
# --- Matcher 의 사용예시 --- # 
# 문법적 오류가 없는 문장, 그리고 의미상 호감을 표현하는 문장만 매치를 해봅시다.
KEY = "a grammatically correct expression of affection"
EG_BATCH = [
            "I love apples",
            "I loved him", 
            "I like nlp",
            "I liked",
            "I loved",
            "I hate them"
]

# 이를 위해 다음과 같은 패턴을 만들어볼 수 있습니다.
patterns = [
  {"TEXT": "I"},  # 첫번째 단어는 반드시 I로 하겠다.
  {"LEMMA": {"IN": ["like", "love"]}},  # 두번째 단어의 표제어는 반드시 like, love 중 하나여야 한다.
  {"POS": {"IN": ["PRON", "NOUN"]}}  # 품사추출의 활용; liked, loved는 타동사 이므로, 세번째 단어로는 반드시 목적어가 와야한다.
]

# vocab을 기억한다. key에 새로운 정수를 부여하기 위해서 nlp.vocab을 인자로 받는다.
love_matcher = Matcher(nlp.vocab)
love_matcher.add(KEY, [patterns]) # 다음의 문서참조; https://spacy.io/api/matcher#add
for sent in EG_BATCH:
  doc = nlp(sent)
  matches = love_matcher(doc)
  if matches:
    match_id, start_idx, end_idx = matches[0] 
    print(match_id)
    # nlp.vocab.strings를 통해, 토큰의 id -> 토큰의 str을 얻을 수 있습니다.
    key = nlp.vocab.strings[match_id]
    # 결과를 확인해볼게요!
    print(doc[start_idx: end_idx], "->", key)
# ---------------------- #


10756009303937775286
I love apples -> a grammatically correct expression of affection
10756009303937775286
I loved him -> a grammatically correct expression of affection
10756009303937775286
I like nlp -> a grammatically correct expression of affection


자, 그럼 이제 본격적으로 `IDIOMS`를 검출할 수 있는 `patterns`를 만들어보겠습니다. 

앞서, 각 관용구를 검출하기 위해선 다음과 같이 패턴을 정의할 수 있을 것이라고 했습니다:
 - `stand on [POS=인칭소유격] own feet`
 - `open [POS=인칭소유격] eyes`
 
아울러, 우리는 동사의 활용형에도 대응을 해야합니다. stand는 standing으로,
 open은 opened로 얼마든지 변형될 수 있습니다. 이 부분은 어떻게 대응할 수 있을까요? 
 - 힌트: 위 `love_matcher`는 어떻게 `liked`, `loved`에 대응할 수 있었나요?

In [11]:
def build_patterns(idiom: str) -> List[dict]:
    # PRP_PLACEHOLDER & nlp는 local variable이 아닌 global variable (전역변수)입니다.
    # 그 점을 상기하기 위해, 그리고 함수내에서 동명의 변수를 만들지 않기 위해 global선언을 해줍시다.
    global PRP_PLACEHOLDER, nlp
    # 각 패턴은 dictionary 객체로 정의됩니다. 모든 패턴을 리스트로 모아주는 것이 목표입니다.
    patterns: List[dict] = list()
    doc = nlp(idiom)
    # --- TODO 1 --- #
    # hint1: 만약 token.text == one's 라면, 패턴을 어떻게 정의해야할까요?
    # hint2: 나머지 단어는 패턴을 어떻게 정의해야할까요? LEMMA를 활용해볼 수 있을까요?
    for token in doc:
      if token.text == PRP_PLACEHOLDER:
        pattern = {'TAG' : 'PRP$'}
      else:
        pattern = {'LEMMA' : token.lemma_}
      patterns.append(pattern)
    return patterns
    # -------------- #

In [12]:
print(build_patterns("stand on one's own feet"))

[{'LEMMA': 'stand'}, {'LEMMA': 'on'}, {'TAG': 'PRP$'}, {'LEMMA': 'own'}, {'LEMMA': 'foot'}]


In [15]:
def build_patterns_list_comp(idiom: str) -> List[dict]:
  global PRP_PLACEHOLDER, nlp
  # --- TODO 2 --- # 
  # list comprehension을 활용하면, build_patterns와 동일한 코드를 한줄로(!?)작성할 수 있습니다.
  # 한번 도전해보시길! 못해도 괜찮습니다. 수업 때 같이 생각해봐요!
  # hint: if-else 문을 list comprehension 속에 집어넣을 수도 있습니다! - https://stackoverflow.com/a/4406399
  doc = nlp(idiom)
  patterns: List[dict] = [
                          {"TAG" : "PRP$"} if token.text == PRP_PLACEHOLDER
                          else {"LEMMA" : token.lemma_}
                          for token in doc
  ]
  return patterns
  # -------------- #

In [16]:
print(build_patterns_list_comp("stand on one's own feet"))

[{'LEMMA': 'stand'}, {'LEMMA': 'on'}, {'TAG': 'PRP$'}, {'LEMMA': 'own'}, {'LEMMA': 'foot'}]


In [21]:
# 이제 여러분이 구현한 함수를 바탕으로, 관용구를 검출할 수 있는 matcher를 만들어보겠습니다!
idiom_matcher = Matcher(nlp.vocab)
for idiom in IDIOMS:
  # idiom_patterns = build_patterns_list_comp(idiom)
  # list comprehension으로 구현하는 것에 성공하셨다면, 아래 함수를 사용해서 테스트해보세요!
  idiom_patterns = build_patterns_list_comp(idiom)
  idiom_matcher.add(idiom, [idiom_patterns])

**Question: 객체 자체를 함수로 쓰는 것은 뭐죠? (e.g. `love_matcher(doc)`, `idiom_matcher(doc` )**
> A: 객체 자체를 호출 할 경우, 해당 객체의 `__call__()` 구현체가 실행이됩니다.
- 즉 [`matcher.__call__()`](https://spacy.io/api/matcher) 이 실행됩니다.
- 애초에 클래스 이름이 `Matcher`이므로, 구태여 `matcher.match()`를 하지 않기 위해 `__call__()` dunder method를 구현한 것으로 예상됩니다.


In [ ]:
# ---  object.__call__() dunder method 의 사용 예시 --- #
class Printer:
  def __call__(self, name: str) -> str:
    return "{} called __call__".format(name)

  def print(self, name: str):
    return "{} called __call__".format(name)

printer = Printer()
# 물론 이렇게 할수도 있지만... 애초에 클래스 이름이 Printer인데, 구태여 print라는 함수를 또 만들기에는 이름이 중복됩니다:
print(printer.print("유빈이"))  
# 그럴바에는 그냥 printer를 호출했을 때 "print"를 하는 것이 더 간결하겠죠: 
print(printer("베프는")) 
# 그리고 말씀드린대로, 객체를 호출하는 것은 결국 __call__()을 호출하는 것과 같습니다:
print(printer.__call__("석신이"))
#--------------------------------------------------- #


유빈이 called __call__
베프는 called __call__
석신이 called __call__


In [22]:
# 이제 준비한 BATCH로 idiom_matcher를 테스트 해봅시다. 
correct = 0
total = len(BATCH)
for eg, idiom in BATCH:
  doc = nlp(eg)
  res = idiom_matcher(doc)
  if res:  # if len(res) > 0와 의미는 같습니다.
    # 한 문장에 여러 관용구가 나올수도 있으므로, res의 타입은 list입니다.
    # 우리의 BATCH 데이터에는 한 문장에 하나의 관용구만 존재하므로, res[0]로 첫번째 결과를 가져와줍니다.
    match_id, start_idx, end_idx = res[0]
    match_idiom = nlp.vocab.strings[match_id]
    # list slicing으로 해당 관용구의 활용형을 가져옵니다.
    print(doc[start_idx:end_idx], "->", match_idiom)
    if match_idiom == idiom:
      correct += 1

# 최대값인 1.0이 나오면 성공입니다!
print("관용구 검출 정확도:", correct / total)


stand on your own feet -> stand on one's own feet
stand on their own feet -> stand on one's own feet
standing on my own feet -> stand on one's own feet
opened my eyes -> open one's eyes
opened her eyes -> open one's eyes
관용구 검출 정확도: 1.0


다음과 같은 결과가 나오면 됩니다:
```
stand on your own feet -> stand on one's own feet
stand on their own feet -> stand on one's own feet
standing on my own feet -> stand on one's own feet
opened my eyes -> open one's eyes
opened her eyes -> open one's eyes
관용구 검출 정확도: 1.0
```

---
모두 수고하셨습니다 :) 오타 정정 및 질의 응답은 DM으로 보내주세요!
